<font size=5><b>Data Analysis of Amazon Top 50 Bestselling Books from 2009-2011 using SQLITE</b></font>

In [1]:
import sqlite3
import pandas as pd

file_path = '/Users/seanbunk/Desktop/Data_Projects/SQLite3/Top50AmazonBooksEDA/bestsellers with categories 2.csv'
data = pd.read_csv(file_path)

connection = sqlite3.connect('my_books.db')

data.to_sql('bestsellers', connection, if_exists= 'replace', index = False)



550

In [2]:
query = 'SELECT * FROM bestsellers'
df = pd.read_sql_query(query, connection)

print(df.head())

                                                Name  \
0                      10-Day Green Smoothie Cleanse   
1                                  11/22/63: A Novel   
2            12 Rules for Life: An Antidote to Chaos   
3                             1984 (Signet Classics)   
4  5,000 Awesome Facts (About Everything!) (Natio...   

                     Author  User Rating  Reviews  Price  Year        Genre  
0                  JJ Smith          4.7    17350      8  2016  Non Fiction  
1              Stephen King          4.6     2052     22  2011      Fiction  
2        Jordan B. Peterson          4.7    18979     15  2018  Non Fiction  
3             George Orwell          4.7    21424      6  2017      Fiction  
4  National Geographic Kids          4.8     7665     12  2019  Non Fiction  


In [3]:
#Check for missing values
query = """ SELECT
    SUM(CASE WHEN Name IS NULL THEN 1 ELSE 0 END) AS Missing_Names,
    SUM(CASE WHEN Author IS NULL THEN 1 ELSE 0 END) AS Missing_Authors,
    SUM(CASE WHEN 'User Rating' IS NULL THEN 1 ELSE 0 END) AS Missing_User_Ratings,
    SUM(CASE WHEN Reviews IS NULL THEN 1 ELSE 0 END) AS Missing_Reviews,
    SUM(CASE WHEN Price IS NULL THEN 1 ELSE 0 END) AS Missing_Prices,
    SUM(CASE WHEN Year IS NULL THEN 1 ELSE 0 END) AS MIssing_Years,
    SUM(CASE WHEN Genre IS NULL THEN 1 ELSE 0 END) AS Missing_Genres
    FROM bestsellers;"""

missing_values = pd.read_sql_query(query, connection)
print(missing_values)

   Missing_Names  Missing_Authors  Missing_User_Ratings  Missing_Reviews  \
0              0                0                     0                0   

   Missing_Prices  MIssing_Years  Missing_Genres  
0               0              0               0  


In [4]:
#removing rows with missing names
remove_missing_name_query = """
DELETE FROM bestsellers
WHERE Name IS NULL;
"""
connection.execute(remove_missing_name_query)
connection.commit()


In [5]:
#Remove Duplicates
remove_duplicates_query = """
DELETE FROM bestsellers
WHERE rowid NOT IN (
    SELECT MIN(rowid)
    FROM bestsellers
    GROUP BY Name, Author, Year, 'User Rating', Reviews, Price, Genre)"""

connection.execute(remove_duplicates_query)
connection.commit()


In [6]:
query = "SELECT * FROM bestsellers LIMIT 10"
clean_data_sample = pd.read_sql_query(query, connection)
print(clean_data_sample)

                                                Name  \
0                      10-Day Green Smoothie Cleanse   
1                                  11/22/63: A Novel   
2            12 Rules for Life: An Antidote to Chaos   
3                             1984 (Signet Classics)   
4  5,000 Awesome Facts (About Everything!) (Natio...   
5      A Dance with Dragons (A Song of Ice and Fire)   
6  A Game of Thrones / A Clash of Kings / A Storm...   
7                     A Gentleman in Moscow: A Novel   
8      A Higher Loyalty: Truth, Lies, and Leadership   
9                          A Man Called Ove: A Novel   

                     Author  User Rating  Reviews  Price  Year        Genre  
0                  JJ Smith          4.7    17350      8  2016  Non Fiction  
1              Stephen King          4.6     2052     22  2011      Fiction  
2        Jordan B. Peterson          4.7    18979     15  2018  Non Fiction  
3             George Orwell          4.7    21424      6  2017      Fic

<font size=5><b>What are the top 5 highest rated books?</b></font>

In [7]:
query = """
SELECT *
FROM bestsellers
ORDER BY 'User Rating' DESC, Reviews DESC 
LIMIT 10;"""

top_rated_books = pd.read_sql_query(query, connection)
print(top_rated_books)

                      Name          Author  User Rating  Reviews  Price  Year  \
0  Where the Crawdads Sing     Delia Owens          4.8    87841     15  2019   
1    The Girl on the Train   Paula Hawkins          4.1    79446     18  2015   
2    The Girl on the Train   Paula Hawkins          4.1    79446      7  2016   
3                 Becoming  Michelle Obama          4.8    61133     11  2018   
4                 Becoming  Michelle Obama          4.8    61133     11  2019   
5                Gone Girl   Gillian Flynn          4.0    57271     10  2012   
6                Gone Girl   Gillian Flynn          4.0    57271     10  2013   
7                Gone Girl   Gillian Flynn          4.0    57271      9  2014   
8   The Fault in Our Stars      John Green          4.7    50482     13  2012   
9   The Fault in Our Stars      John Green          4.7    50482     13  2013   

         Genre  
0      Fiction  
1      Fiction  
2      Fiction  
3  Non Fiction  
4  Non Fiction  
5     

<font size=3><b>Answer:  The top 5 books are "Where the Crawdads Sing", 
    "The Girl on the Train", "Becoming Michelle Obama", "Gone Girl", and last "The Fault in Our Stars".</b></font>

<font size=5><b>Which authors have the most books in the bestsellers list?</b></font>

In [8]:
query = """
SELECT Author, COUNT(*) as 'Number of Books'
FROM bestsellers
GROUP BY Author
ORDER BY 'Number of Books' DESC
LIMIT 5"""

most_books_authors = pd.read_sql_query(query, connection)
print(most_books_authors)

             Author  Number of Books
0      Zhi Gang Sha                2
1  Wizards RPG Team                3
2  William P. Young                2
3     William Davis                2
4   Walter Isaacson                3


<font size= 3><b>Answer: Zhi Gang Sha, Wizards RPG Team, William p. Young, William Davis, and Walter Issacson</b></font>

<font size=5><b> How does the average user vary between Fiction and Non-Fiction?</b></font>

In [9]:

query = """
SELECT Genre, AVG(`User Rating`) as `Average Rating`
FROM bestsellers
GROUP BY Genre;
"""

avg_rating_genre = pd.read_sql_query(query, connection)
print(avg_rating_genre)


         Genre  Average Rating
0      Fiction        4.648333
1  Non Fiction        4.595161


<font size=3><b>Answer: The average rating for the Fiction genre is 4.6, and the average rating for the Non-Fiction genre</b></font>

<font size=5><b>What is the average price of books per year?</b></font>

In [10]:
query = """
SELECT Year, AVG(Price) as `Average Price`
FROM bestsellers
GROUP BY Year
ORDER BY Year;
"""
avg_price_year = pd.read_sql_query(query, connection)
print(avg_price_year)

    Year  Average Price
0   2009          15.40
1   2010          13.48
2   2011          15.10
3   2012          15.30
4   2013          14.60
5   2014          14.64
6   2015          10.42
7   2016          13.18
8   2017          11.38
9   2018          10.52
10  2019          10.08


<font size=5><b>What is the number of best_selling books per genre each year?</b></font>

In [11]:
query = """
SELECT Year, Genre, COUNT(*) as 'Number of Books'
FROM bestsellers
GROUP BY Year, Genre
ORDER By Year, Genre;"""

books_per_genre_year = pd.read_sql_query(query, connection)
print(books_per_genre_year)

    Year        Genre  Number of Books
0   2009      Fiction               24
1   2009  Non Fiction               26
2   2010      Fiction               20
3   2010  Non Fiction               30
4   2011      Fiction               21
5   2011  Non Fiction               29
6   2012      Fiction               21
7   2012  Non Fiction               29
8   2013      Fiction               24
9   2013  Non Fiction               26
10  2014      Fiction               29
11  2014  Non Fiction               21
12  2015      Fiction               17
13  2015  Non Fiction               33
14  2016      Fiction               19
15  2016  Non Fiction               31
16  2017      Fiction               24
17  2017  Non Fiction               26
18  2018      Fiction               21
19  2018  Non Fiction               29
20  2019      Fiction               20
21  2019  Non Fiction               30


In [12]:
connection.close()

<font size=5><b>Thank you for viewing my Data Analysis of Top 50 best selling books on Amazon from 2009 - 2011.</b></font>